In [1]:
import commit
from commit import trk2dictionary
from os.path import join
import numpy as np
commit.setup()


# command to run tractography
sub_folder='/home/pabaua/dev_mni/data/sub-PNC001_ses-01/sub-PNC001/ses-01/dwi' \
sub='sub-PNC001_ses-01' \
tracts='10000' \
tck="${sub_folder}/${sub}_space-dwi_desc-iFOD2-${tracts}_tractography.tck" \
tckgen \
        ${sub_folder}/sub-PNC001_ses-01_space-dwi_model-CSD_map-FOD_desc-wmNorm_peaks.nii.gz \
        "$tck" \
        -act ${sub_folder}/sub-PNC001_ses-01_space-dwi_desc-5tt.nii.gz \
        -crop_at_gmwmi \
        -backtrack \
        -seed_dynamic ${sub_folder}/sub-PNC001_ses-01_space-dwi_model-CSD_map-FOD_desc-wmNorm_peaks.nii.gz \
        -algorithm iFOD2 \
        -step 0.5 \
        -angle 22.5 \
        -cutoff 0.06 \
        -maxlength 400 \
        -minlength 10 \
        -select "$tracts"

In [2]:

trk2dictionary.run(
    filename_tractogram = 'demo01_fibers.tck',
    filename_peaks      = 'peaks.nii.gz',
    filename_mask       = 'WM.nii.gz',
    fiber_shift         = 0.5,
    peaks_use_affine    = True
)


-> Precomputing rotation matrices:
   [ DONE ]                                                                                                                                          


In [17]:
# Prepare acquisition scheme information in FSL format
# `mrconvert -export_grad_fsl <bvec_output_file> <bval_output_file> <mrtrix_input_file> <nifti_output_file>`
NODDI_bval = join(ae.get_config("study_path"), ae.get_config("subject"), 'ses-01/dwi/sub-PNC001_ses-01_space-dwi_desc-preproc_dwi.bval')
NODDI_bvec = join(ae.get_config("study_path"), ae.get_config("subject"), 'ses-01/dwi/sub-PNC001_ses-01_space-dwi_desc-preproc_dwi.bvec')
amico.util.fsl2scheme(NODDI_bval, NODDI_bvec)


-> Writing scheme file to [ /home/pabaua/dev_mni/data/sub-PNC001_ses-01/sub-PNC001/ses-01/dwi/sub-PNC001_ses-01_space-dwi_desc-preproc_dwi.scheme ]


'/home/pabaua/dev_mni/data/sub-PNC001_ses-01/sub-PNC001/ses-01/dwi/sub-PNC001_ses-01_space-dwi_desc-preproc_dwi.scheme'

In [18]:
# Load the diffusion signal and its corresponding acquisition scheme.
NODDI_img = join(ae.get_config("study_path"), ae.get_config("subject"), 'ses-01/dwi/sub-PNC001_ses-01_space-dwi_desc-preproc_dwi.nii.gz')
NODDI_scheme = join(ae.get_config("study_path"), ae.get_config("subject"), 'ses-01/dwi/sub-PNC001_ses-01_space-dwi_desc-preproc_dwi.scheme')
brain_mask = join(ae.get_config("study_path"), ae.get_config("subject"), 'ses-01/dwi/sub-PNC001_ses-01_space-dwi_desc-brain_mask.nii.gz')
ae.load_data(NODDI_img, NODDI_scheme, mask_filename=brain_mask, b0_thr=50)


-> Loading data:
	* DWI signal
		- dim    = 192 x 192 x 132 x 143
		- pixdim = 1.099 x 1.099 x 1.100
	* Acquisition scheme
		- 143 samples, 28 shells
		- 3 @ b=0 , 4 @ b=2010.0 , 8 @ b=2000.0 , 7 @ b=1970.0 , 11 @ b=1980.0 , 8 @ b=1975.0 , 7 @ b=2005.0 , 4 @ b=2030.0 , 5 @ b=1990.0 , 7 @ b=2015.0 , 6 @ b=2025.0 , 9 @ b=1995.0 , 7 @ b=1985.0 , 5 @ b=2020.0 , 2 @ b=1965.0 , 3 @ b=305.0 , 2 @ b=300.0 , 1 @ b=310.0 , 1 @ b=295.0 , 3 @ b=290.0 , 8 @ b=715.0 , 5 @ b=705.0 , 7 @ b=700.0 , 5 @ b=695.0 , 5 @ b=710.0 , 4 @ b=690.0 , 1 @ b=720.0 , 4 @ b=685.0 , 1 @ b=680.0 
	* Binary mask
		- dim    = 192 x 192 x 132
		- pixdim = 1.099 x 1.099 x 1.100
		- voxels = 915927
   [ 23.4 seconds ]

-> Preprocessing:
	* Normalizing to b0... [ min=-451.24,  mean=0.68, max=1028.70 ]
	* Keeping all b0 volume(s)
   [ 6.8 seconds ]


In [22]:
# Set the model to use to describe the signal contributions in each voxel.
# models: ['StickZeppelinBall', 'CylinderZeppelinBall', 'NODDI', 'FreeWater', 'SANDI']
ae.set_model('NODDI')

# Define NODDI model parameters to compute each compartment response function
# para_diff is the axial diffusivity (AD) in the CC -- single fiber
para_diff=1.7E-3
# iso_diff is the mean diffusivity (MD) in ventricles.
iso_diff=3.0E-3
intra_vol_frac = np.linspace(0.1, 0.99, 12)
intra_orient_distr = np.hstack((np.array([0.03, 0.06]), np.linspace(0.09, 0.99, 10)))
ae.model.set(dPar=para_diff, dIso=iso_diff,IC_VFs=intra_vol_frac, IC_ODs=intra_orient_distr, isExvivo=False)

# Generate the high-resolution response functions for each compartment with:
# lambda1 is the first regularization parameter.
# lambda2 is the second regularization parameter.
#        StickZeppelinBall:      'set_solver()' not implemented
#        CylinderZeppelinBall:   lambda1 = 0.0, lambda2 = 4.0
#        NODDI:                  lambda1 = 5e-1, lambda2 = 1e-3
#        FreeWater:              lambda1 = 0.0, lambda2 = 1e-3
#        VolumeFractions:        'set_solver()' not implemented
#        SANDI:                  lambda1 = 0.0, lambda2 = 5e-3
ae.set_solver(lambda1=5e-1, lambda2=1e-3)
ae.generate_kernels(regenerate=True)


-> Creating LUT for "NODDI" model:
   [ 28.0 seconds ]                                                                                                                                  


In [16]:
# Load rotated kernels and project to the specific gradient scheme of this subject.
ae.load_kernels()
# Fit the model to the data.
ae.fit()
# Save the output (directions, maps etc).
ae.save_results()


-> Resampling LUT for subject ".":
   [ 2.9 seconds ]                                                                                                                                   

-> Estimating principal directions (OLS):
   [ 00h 00m 05s ]                                                                                                                                   

-> Fitting 'NODDI' model to 915927 voxels (using 8 threads):
   [ 00h 11m 01s ]                                                                                                                                   
